In [1]:
import numpy as np 
import seaborn as sns
import pandas as pd 
pd.options.plotting.backend = "plotly"
import plotly.graph_objects as go
import plotly.express as px
import yfinance as yf
import quandl
from fredapi import Fred

pd.set_option("display.max_rows",200)
sns.set(rc={'figure.figsize':(16,10)})
fred_key = "df4910b2cad947d95cf6ab16ba11d74d"
fred = Fred(api_key = fred_key)
quandl.ApiConfig.api_key = 'Qq5R29Xiqp2yUbb9dzNq'

# geting the macroeconomics code and there transformations

In [2]:
MacroCode=pd.read_csv(r'Data\Macro Variables.csv')
MacroCode.replace({'Average HOUST':'HOUST','S&P 500':'SP500'},inplace=True)

In [3]:
errors=[]
indicators=[]

for code in MacroCode['Variable']:
    if not ('S&P' in code):
        try:
            indicators.append(fred.get_series(code).to_frame(code))
        except ValueError:
            errors.append(code)
indicators.append(yf.download('^SP500-20')['Close'].to_frame('S&P: indust'))
indicators.append(quandl.get("MULTPL/SP500_DIV_YIELD_MONTH").squeeze().to_frame('S&P div yield'))
indicators.append(quandl.get("MULTPL/SP500_PE_RATIO_MONTH").squeeze().to_frame('S&P PE ratio'))


[*********************100%***********************]  1 of 1 completed

1 Failed download:
- ^SP500-20: No data found for this date range, symbol may be delisted


In [4]:
if len(errors)>0:
    raise Exception("Erorrs found")
data=pd.concat(indicators,axis=1).fillna(method='ffill')
# data.to_csv('Data/HistoricalVariables.csv')


# Data transformation

In [5]:
def deltaX(data):
    return data.diff()

def delta2X(data):
    return data-2*data.shift(1)+data.shift(2)

def logdata(data):
    return np.log(data)

def difflog(data):
    return np.log(data).diff()

def difflog2(data):
    return np.log(data)-2*np.log(data.shift(1))+np.log(data.shift(2))

def diffpercent(data):
    return (data/data.shift() - 1) - (data.shift()/data.shift(2) - 1)



transformation={1:(lambda x: x),
                2:deltaX,
                3:delta2X,
                4:logdata,
                5:difflog,
                6:difflog2,
                7:diffpercent,
                }

In [6]:
df_transformed=pd.DataFrame()

In [7]:
for column in data.columns:
    type=MacroCode[MacroCode['Variable'] == column]['Transformation'].iloc[0]
    df_transformed[column]=transformation[type](data[column])
    df_transformed=df_transformed.copy()